# Here’s a road map of what we will do with Pandas:

1. Set up the environment and load the data
2. Investigate the data
3. Parse the different data tabs
4. Standardize existing columns and create new ones
5. Clean up the data using “apply” and “lambda” functions
3. Reshape the data from wide to long by pivoting on multi-level indices and stacking
3. Save the final results back to excel





# 1. Set up the environment and load the data

In [47]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/reshaping-data/reshaping_data.xlsx


In [48]:
# Load the raw data using the ExcelFile object
data = pd.ExcelFile('/kaggle/input/reshaping-data/reshaping_data.xlsx')

# 2. Investigate the data

In [49]:
# First, see the sheetnames available
data.sheet_names

['ABC_inc', 'HIJ_inc', 'OPQ_inc', 'XYZ_inc']

In [50]:
# Take a peek at the first 10 rows of the first tab
data.parse(sheetname='ABC_inc', skiprows=0).head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Year1,...,NaN,NaN,Year4,NaN,NaN,NaN,NaN,Year5,NaN,NaN
6,NaN,district,province,partner,funding_source,NaN,NaN,2017,2017,2017,...,NaN,2020,2020,2020,2020,NaN,2021,2021,2021,2021
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10-14yrs,15-29yrs,30+yrs,...,NaN,10-14yrs,15-29yrs,30+yrs,Total,NaN,10-14yrs,15-29yrs,30+yrs,Total
8,NaN,District 1,Region 1,partner 1,Souce 2,NaN,NaN,1296,383,1571,...,NaN,1906,1925,931,5465,NaN,61,353,1091,2409
9,NaN,District 2,Region 3,partner 6,Souce 5,NaN,NaN,722,232,1848,...,NaN,810,664,452,3665,NaN,989,374,1790,4320


In [51]:
tabnames = data.sheet_names

In [52]:
i = 0
df = data.parse(sheetname=tabnames[i], skiprows=7)
df.head(2)

,Unnamed: 0,district,province,partner,funding_source,Unnamed: 5,Unnamed: 6,2017,2017.1,2017.2,...,Unnamed: 21,2020,2020.1,2020.2,2020.3,Unnamed: 26,2021,2021.1,2021.2,2021.3
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10-14yrs,15-29yrs,30+yrs,...,NaN,10-14yrs,15-29yrs,30+yrs,Total,NaN,10-14yrs,15-29yrs,30+yrs,Total
1,NaN,District 1,Region 1,partner 1,Souce 2,NaN,NaN,1296,383,1571,...,NaN,1906,1925,931,5465,NaN,61,353,1091,2409


# 4. Standardize existing columns and create new ones

In [53]:
# make a list of the header row and strip up to the 4th letter. This is the location and year information
cols1 = list(df.columns)
cols1 = [str(x)[:4] for x in cols1]

# make another list of the first row,this is the age group information
# we need to preserve this information in the column name when we reshape the data 
cols2 = list(df.iloc[0,:])
cols2 = [str(x) for x in cols2]

# now join the two lists to make a combined column name which preserves our location, year and age-group information
cols = [x+"_"+y for x,y in zip(cols1,cols2)]

# Assign new column names to the dataframe
df.columns = cols
df.head(1)

,Unna_nan,dist_nan,prov_nan,part_nan,fund_nan,Unna_nan,Unna_nan,2017_10-14yrs,2017_15-29yrs,2017_30+yrs,...,Unna_nan,2020_10-14yrs,2020_15-29yrs,2020_30+yrs,2020_Total,Unna_nan,2021_10-14yrs,2021_15-29yrs,2021_30+yrs,2021_Total
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10-14yrs,15-29yrs,30+yrs,...,NaN,10-14yrs,15-29yrs,30+yrs,Total,NaN,10-14yrs,15-29yrs,30+yrs,Total


In [54]:
# Drop empty columns, Rename the useful columns
# Note when you drop, you should specify axis=1 for columns and axis=0 for rows
df = df.drop(["Unna_nan"], axis=1).iloc[1:,:].rename(columns={'dist_nan':'district', 'prov_nan': 'province', 'part_nan':'partner', 'fund_nan':'financing_source'})
df.head(2)

,district,province,partner,financing_source,2017_10-14yrs,2017_15-29yrs,2017_30+yrs,2017_Total,2018_10-14yrs,2018_15-29yrs,...,2019_30+yrs,2019_Total,2020_10-14yrs,2020_15-29yrs,2020_30+yrs,2020_Total,2021_10-14yrs,2021_15-29yrs,2021_30+yrs,2021_Total
1,District 1,Region 1,partner 1,Souce 2,1296,383,1571,3250,189,854,...,491,2256,1906,1925,931,5465,61,353,1091,2409
2,District 2,Region 3,partner 6,Souce 5,722,232,1848,2802,972,69,...,245,2957,810,664,452,3665,989,374,1790,4320


In [55]:
# Engineer a new column for the organization, grab this name from the excel tab name
# This should read 'ABC inc' if executed correctly
df['main_organization'] = tabnames[i].split("_")[0] + " "+ tabnames[i].split("_")[1]
df.main_organization.head(2)

1    ABC inc
2    ABC inc
Name: main_organization, dtype: object

**Let’s pause and look at the structure of our dataframe so far.**

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 1 to 10
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   district           10 non-null     object
 1   province           10 non-null     object
 2   partner            10 non-null     object
 3   financing_source   10 non-null     object
 4   2017_10-14yrs      10 non-null     object
 5   2017_15-29yrs      10 non-null     object
 6   2017_30+yrs        10 non-null     object
 7   2017_Total         10 non-null     object
 8   2018_10-14yrs      10 non-null     object
 9   2018_15-29yrs      10 non-null     object
 10  2018_30+yrs        10 non-null     object
 11  2018_Total         10 non-null     object
 12  2019_10-14yrs      10 non-null     object
 13  2019_15-29yrs      10 non-null     object
 14  2019_30+yrs        10 non-null     object
 15  2019_Total         10 non-null     object
 16  2020_10-14yrs      10 non-null     object
 17 

# 5. Clean up the data using “apply” and “lambda” functions

In [57]:
# Make lists of the columns which need attention and use this as reference to execute
# You will notice that I use list comprehension every time I generate an iterable like a list or dictionary
# This is really amazing python functionality and I never want to go back to the old looping way of doing this!
to_remove = [c for c in df.columns if "Total" in c] # redundant
to_change = [c for c in df.columns if "yrs" in c] # numeric

# drop unwanted columns
# Notice that you need to specify inplace, otherwise pandas will return the data frame instead of changing it in place
df.drop(to_remove, axis=1, inplace= True) 

# Change the target column data types
for c in to_change:
    df[c] = df[c].apply(lambda x: pd.to_numeric(x))

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 1 to 10
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   district           10 non-null     object
 1   province           10 non-null     object
 2   partner            10 non-null     object
 3   financing_source   10 non-null     object
 4   2017_10-14yrs      10 non-null     int64 
 5   2017_15-29yrs      10 non-null     int64 
 6   2017_30+yrs        10 non-null     int64 
 7   2018_10-14yrs      10 non-null     int64 
 8   2018_15-29yrs      10 non-null     int64 
 9   2018_30+yrs        10 non-null     int64 
 10  2019_10-14yrs      10 non-null     int64 
 11  2019_15-29yrs      10 non-null     int64 
 12  2019_30+yrs        10 non-null     int64 
 13  2020_10-14yrs      10 non-null     int64 
 14  2020_15-29yrs      10 non-null     int64 
 15  2020_30+yrs        10 non-null     int64 
 16  2021_10-14yrs      10 non-null     int64 
 17 

# 6. Reshape the data from wide to long by pivoting on multi-level indices and stacking

In [59]:
# First, select the columns to use for a multi-level index. This depends on your data
# Generally, you want all the identifier columns to be included in the multi-index 
# For this dataset, this is every non-numeric column
idx =['district', 'province', 'partner', 'financing_source', 'main_organization']

# Then pivot the dataset based on this multi-level index 
multi_indexed_df = df.set_index(idx)
multi_indexed_df.head(2)

,,,,,2017_10-14yrs,2017_15-29yrs,2017_30+yrs,2018_10-14yrs,2018_15-29yrs,2018_30+yrs,2019_10-14yrs,2019_15-29yrs,2019_30+yrs,2020_10-14yrs,2020_15-29yrs,2020_30+yrs,2021_10-14yrs,2021_15-29yrs,2021_30+yrs
district,province,partner,financing_source,main_organization,,,,,,,,,,,,,,,
District 1,Region 1,partner 1,Souce 2,ABC inc,1296,383,1571,189,854,339,840,773,491,1906,1925,931,61,353,1091
District 2,Region 3,partner 6,Souce 5,ABC inc,722,232,1848,972,69,1205,422,676,245,810,664,452,989,374,1790


In [60]:
# Stack the columns to achieve the baseline long format for the data
stacked_df = multi_indexed_df.stack(dropna=False)
stacked_df.head(25) 

# check out the results!

district    province  partner    financing_source  main_organization               
District 1  Region 1  partner 1  Souce 2           ABC inc            2017_10-14yrs    1296
                                                                      2017_15-29yrs     383
                                                                      2017_30+yrs      1571
                                                                      2018_10-14yrs     189
                                                                      2018_15-29yrs     854
                                                                      2018_30+yrs       339
                                                                      2019_10-14yrs     840
                                                                      2019_15-29yrs     773
                                                                      2019_30+yrs       491
                                                                      2020_10-14yrs    1

**Mind. Blown! Exactly how I felt when I saw this for the first time too.**

In [61]:
# Now do a reset to disband the multi-level index, we only needed it to pivot our data during the reshape
long_df = stacked_df.reset_index()
long_df.head(3)

,district,province,partner,financing_source,main_organization,level_5,0
0,District 1,Region 1,partner 1,Souce 2,ABC inc,2017_10-14yrs,1296
1,District 1,Region 1,partner 1,Souce 2,ABC inc,2017_15-29yrs,383
2,District 1,Region 1,partner 1,Souce 2,ABC inc,2017_30+yrs,1571


In [62]:
# Make series of lists which split year from target age-group
# the .str attribute is how you manipulate the data frame objects and columns with strings in them
col_str = long_df.level_5.str.split("_") 
col_str.head(3)

0    [2017, 10-14yrs]
1    [2017, 15-29yrs]
2      [2017, 30+yrs]
Name: level_5, dtype: object

In [63]:
# engineer the columns we want, one columns takes the first item in col_str and another columns takes the second 
long_df['target_year'] = [x[0] for x in col_str] 
long_df['target_age'] = [x[1] for x in col_str]
long_df['target_quantity'] = long_df[0] # rename this column
long_df.head(2)

,district,province,partner,financing_source,main_organization,level_5,0,target_year,target_age,target_quantity
0,District 1,Region 1,partner 1,Souce 2,ABC inc,2017_10-14yrs,1296,2017,10-14yrs,1296
1,District 1,Region 1,partner 1,Souce 2,ABC inc,2017_15-29yrs,383,2017,15-29yrs,383


In [64]:
# drop the now redundant columns
df_final = long_df.drop(['level_5', 0], axis=1)
df_final.head(2)

,district,province,partner,financing_source,main_organization,target_year,target_age,target_quantity
0,District 1,Region 1,partner 1,Souce 2,ABC inc,2017,10-14yrs,1296
1,District 1,Region 1,partner 1,Souce 2,ABC inc,2017,15-29yrs,383


# 7. Save the final results back to Excel

In [65]:
df_final.to_excel("reshaping_result_long_format.xlsx")